In [1]:
! pip install emoji contractions

In [1]:
import pandas as pd
from tqdm.notebook import trange, tqdm

In [2]:
data1 = pd.read_csv('isarcasm_train.csv')
data2 = pd.read_csv('sarcasmheadlines_train.csv', index_col=0)
data3 = pd.read_csv('semeval2018_train.csv', index_col=0)
data4 = pd.read_csv('isarcasmeval_train.csv', usecols=['tweet', 'sarcastic'])
data4 = data4.rename(columns={"tweet": "text", "sarcastic": "sarcasm_label"})

data = pd.concat([data1, data2, data3, data4], ignore_index=True)
data

,text,sarcasm_label
0,sorry but sue from the mediweight advert looks...,0
1,@davesnyder since this is fantasy can it be an...,0
2,"Good times, fun times here in Grand old Britan...",1
3,I send light and love to all impacted by the #...,0
4,Another year closer to death,0
...,...,...
38713,The population spike in Chicago in 9 months is...,0
38714,You'd think in the second to last English clas...,0
38715,I’m finally surfacing after a holiday to Scotl...,0
38716,Couldn't be prouder today. Well done to every ...,0


In [3]:
# replace user tags with @user, consecutive whitespaces to single space,
# surround punctuations with space as in https://aclanthology.org/2022.semeval-1.133.pdf
# also replace emojis with text desc using 'emoji'
# !pip install emoji contractions

import re
import emoji
import contractions

def expand_contractions(text):
    expanded_text = contractions.fix(text)
    return expanded_text

def preprocess_tweet(tweet):
  # Replace tagged users with @user
  tweet = re.sub(r'@[\w]+', '@user', str(tweet))

  # Replace multiple white spaces with a single space
  tweet = re.sub(r'\s+', ' ', tweet)

  # Remove more than two successive occurrences of any punctuation
  tweet = re.sub(r'(\s*([.,!?;:()<>{}[\]\-+=~_$%^&]\s){2,})', r'\1', tweet)

  # Replace emoji with emoji descriptor
  tweet = emoji.demojize(tweet)

  # Put a single white space around punctuations
  tweet = re.sub(r'([.,!?;:()<>{}[\]\-+=~_$%^&*])', r' \1 ', tweet)

  # expand contractions
  tweet = expand_contractions(tweet)

  return tweet

In [4]:
data['text'] = data['text'].apply(preprocess_tweet)
data

,text,sarcasm_label
0,sorry but sue from the mediweight advert looks...,0
1,@user since this is fantasy can it be any indi...,0
2,"Good times , fun times here in Grand old Brit...",1
3,I send light and love to all impacted by the #...,0
4,Another year closer to death,0
...,...,...
38713,The population spike in Chicago in 9 months is...,0
38714,You would think in the second to last English ...,0
38715,I am finally surfacing after a holiday to Scot...,0
38716,Could not be prouder today . Well done to eve...,0


In [5]:
# from transformers import pipeline
# import numpy as np
# import torch

# device = 0 if torch.cuda.is_available() else 'cpu'

# model_name = "EleutherAI/gpt-neo-1.3B"
# zero_shot_classifier = pipeline("zero-shot-classification", model=model_name, device=device)

# prompt = '''Here are 4 examples of Sarcastic and Not Sarcastic sentences:
#             Sentence: The only thing I got from college is a caffeine addiction
#             Output: Sarcastic
            
#             Sentence: I love it when professors draw a big question mark next to my answer on an exam because I am always like yeah I do not either ¯\ _  ( ツ )  _ /¯ 
#             Output: Sarcastic

#             Sentence: 2br apt ,  Notts city centre https : //t . co/yFj989BdXR
#             Output: Not Sarcastic
            
#             Sentence: Motivation Wednesday Night :  I AM the powerhouse of the cell .
#             Output: Not Sarcastic
            
#             For the task of Sarcasm Detection, choose whether the following sentence is Sarcastic OR Not Sarcastic. \n'''

# candidate_labels = ['Sarcastic','Not Sarcastic']

# count = 0
# for i in trange(len(data)):
#   input_text = data['text'][i]
#   true = data['sarcasm_label'][i]
  
#   result = zero_shot_classifier(prompt+input_text, candidate_labels)
#   pred = result['labels'][np.argmax(result['scores'])]

#   if pred == 'Sarcastic' and true == 1:
#     count+=1
#   elif pred == 'Not Sarcastic' and true == 0:
#     count+=1
#   print(np.argmax(result['scores']))
#   print(result)
#   print(pred)

## LLM = T5

In [6]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_name = "t5-base"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/home/mila/s/subhrajyoti.dasgupta/.conda/envs/quick_recipe/lib/python3.9/site-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [7]:
prompt = '''Here are 4 examples of Sarcastic and Not Sarcastic sentences:\n 
            Sentence: The only thing I got from college is a caffeine addiction
            Output: True
            
            Sentence: I love it when professors draw a big question mark next to my answer on an exam because I am always like yeah I do not either ¯\ _  ( ツ )  _ /¯ 
            Output: True

            Sentence: 2br apt ,  Notts city centre https : //t . co/yFj989BdXR
            Output: False
            
            Sentence: Motivation Wednesday Night :  I AM the powerhouse of the cell .
            Output: False
            
            For the task of Sarcasm Detection, choose whether the following sentence is Sarcastic OR Not Sarcastic. \n'''


count = 0
for i in trange(len(data)):
  input_text = data['text'][i]
  input_ids = tokenizer.encode(prompt + input_text, return_tensors="pt")
  output_ids = model.generate(input_ids)
  output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
  # print(output_text)
  if output_text and data['sarcasm_label'][i] == 1:
    count+=1

print(count)

  0%|          | 0/38718 [00:00<?, ?it/s]

/home/mila/s/subhrajyoti.dasgupta/.conda/envs/quick_recipe/lib/python3.9/site-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
2023-12-19 06:20:03.208434: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-19 06:20:03.251917: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the app

16883


In [8]:
print(count/len(data))

0.4360504158272638
